In [18]:
import numpy as np
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from scipy.spatial.distance import pdist
from sklearn.neighbors import NearestNeighbors, KDTree
from sklearn.metrics.pairwise import rbf_kernel

In [56]:
num_data = 800
num_samples = num_data // 2
num_features = 2
num_centers = 4
rand_seed = 100

In [57]:
np.random.seed(rand_seed)
data = np.random.rand(num_data, num_features)

In [58]:
# import scipy.io
# mat = scipy.io.loadmat('Syndata.mat')
# data = mat['data']

In [59]:
# K = rbf_kernel(data, data, 1.8)
# print(f"K: {K.shape}")
# np.dot(K[k, :], K[:, k]) / (K[k, k] + lambda_)

In [60]:
# make some fake data X and y
np.random.seed(0)
train_X = np.random.rand(100, 3)
train_y = np.random.randint(0,10, size=100)
# print(train_y.min(), train_y.max())
svc = SVC()
svc.fit(train_X, train_y)

SVC()

In [61]:
test_X = np.random.rand(1, 3)
svc.predict(test_X)

array([2])

In [62]:
# def rbf_kernel(X, Y, sigma):
#     N, K = X.shape
#     M = Y.shape[0]

#     K_xy = np.ones(M)*np.sum(X**2) + np.ones(N)*np.sum(Y**2) - 2*np.dot(X, Y.transpose())
#     K_xy = np.exp(-0.5 * K_xy / sigma**2)

#     return K_xy

In [63]:
# data = np.array(([1,1],[1,2],[3,3]))
K = rbf_kernel(data, data, 1.8)
print(f"K: {K.shape}")
print(K[:10])
# print(K.all(0))

K: (800, 800)
[[1.         0.54721612 0.56746044 ... 0.99036409 0.79249451 0.97405525]
 [0.54721612 1.         0.28403243 ... 0.62878715 0.27846726 0.67467183]
 [0.56746044 0.28403243 1.         ... 0.53710328 0.25922009 0.59177352]
 ...
 [0.34257765 0.89784338 0.24527744 ... 0.40719282 0.12748867 0.46537413]
 [0.86075932 0.33800734 0.30829084 ... 0.83046904 0.99057702 0.75734021]
 [0.87458274 0.42212902 0.2931007  ... 0.86689002 0.95810446 0.79805147]]


In [64]:
def halving(K, m, candidate_index=None, lambda_=0.001):
    
    n = K.shape[0]
    print(f'number of data: {n}')

    m = min(n, m)
    print(f'number of samples: {m}')

    if candidate_index is None:
        candidate_index = np.array(range(n))
    
    # print(f'candidate_index: {candidate_index}')

    q = len(candidate_index)

    index = np.empty(m, dtype=int)
    # print(f'number of index: {index.shape}')

    print('Selecting samples......')
    for i in range(m):
        score = np.zeros(q)
        for j in range(q):
            k = candidate_index[j]
            # print(k)
            score[j] = np.dot(K[k, :], K[:, k]) / (K[k, k] + lambda_)
        
        I = score.argmax()
        # print(I)
        index[i] = candidate_index[I]

        # update K
        # K = K - np.dot(K[:, index[i]], K[index[i], :]) / (K[index[i], index[i]] + lambda_)
        # K = K - K[:, index[i]][:, np.newaxis] * K[index[i], :][np.newaxis, :] / (K[index[i], index[i]] + lambda_)
        K = K - np.outer(K[:, index[i]], K[index[i], :]) / (K[index[i], index[i]] + lambda_)
        # print((np.outer(K[:, index[i]], K[index[i], :])).shape)
        # print((np.outer(K[:, index[i]], K[index[i], :]))==(K[:, index[i]][:, np.newaxis] * K[index[i], :][np.newaxis, :]))

    print('Done.\n')
    return index

In [66]:
K_random = np.random.rand(100, 100)
# print(K_random[:10])
id = halving(K, num_samples)
print(id)
print(len(set(id)))

number of data: 800
number of samples: 400
Selecting samples......
Done.

[351 387 580 735 727 268 317 144 195 331 271 563 200 692 785  69 103 766
 245 509 402 480 307 595 198 441  22 176 349 646 709  98 444 759 570 660
 525  94 168 224  73 474 662 730 727 370  13 213  41 412 199 122 757 482
  94 343  80 234 546 705 759 225 238 388   2 795 133 433 218  61 539 754
 247 150 662 507 474 313 349 480 431  86 282 744 628 428 618 382 560  69
 615 331 223  73 493 412 233 103 153 111  59 513   2 638 225 355 161  61
 150 270 696 435 374 238 328 262 531 234 507 415 660 281 294 416 108 493
 482 713   5 103 399 296 253  33  73 603 329 412 176 570 281  42  13  41
 615 270 320  61 268 696  69   4 612 225 480 507  33 323 662 744  20 166
 413 349 234 385 759 233 238 192 482 466 158 631  33   2   8 228 677 660
 518 696 152 570 424 331  73 665 412 373 685 101 768 759 766 143 234 268
  83 317 744 734  44 507 509 480 662 258 152 273 323 176 474 573 225  15
 238 335 229 349 345 413 139  41 267 329  69  61 2

In [67]:
def number_density(data, center, radius):
    # print(f'length of data: {len(data)}\nlength of center: {len(center)}')
    f = 0
    for i in range(len(data)):
        ball_dist = np.zeros(len(center))
        dist = np.ones(len(center))
        for j in range(len(center)):
            dist[j] = np.linalg.norm(data[i, :] - center[j, :])
            if dist[j] < radius:
                ball_dist[j] = dist[j]

        # print(np.exp(ball_dist/1.8))
        f += np.sum(np.exp(ball_dist/1.8)**2) / (len(ball_dist) + 1)
    
    return f

In [68]:
# kmeans = KMeans(n_clusters=num_centers).fit(data)
# center = kmeans.cluster_centers_
# f = number_density(data, center, radius=0.25)
# f

In [69]:
def SDAL(data, k):

    kmeans = KMeans(n_clusters=k).fit(data)
    center = kmeans.cluster_centers_

    radius = 0.25
    L, R = data.shape
    
    f = number_density(data, center, radius)
    T = 0
    while T<50:
        for j in range(k):
            ball = []
            dist = np.empty(L)
            for i in range(L):
                dist[i] = np.linalg.norm(data[i] - center[j])
                if dist[i] < radius:
                    ball.append(data[i])
            if len(ball)==0:
                center[j] = center[j]
            else:
                center[j] = np.mean(ball)

        F = number_density(data, center, radius)
        
        if F-f==0 or len(np.argwhere(pdist(center)<2*radius))>0:
            break
        else:
            f = F
        T+=1
        radius*=1.1
    
    tree = KDTree(data)
    _, idx = tree.query(center, k=1)
    # print(idx)
    center = data[idx].squeeze()
            
    return center

In [70]:
center = SDAL(data[id], num_centers)
center

array([[0.2043011 , 0.23228687],
       [0.78667137, 0.77845701],
       [0.50181359, 0.50172612],
       [0.50181359, 0.50172612]])